# Lab Activity: Working with Mistrtal
## DSAI4201 Selected Topics in Data Science 5/6
Lynn Younes 60107070

# Step 1: Setup & Install Dependencies

In [12]:
!pip install mistralai
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


# Load API Key (SAFE WAY)

In [2]:
import os

# 🔐 Replace with your real key or set it in Colab Secrets / environment
os.environ["MISTRAL_API_KEY"] = "bpV8Ef5PM9nIQJtEoBcDxyZqNZjxrYes"

api_key = os.getenv("MISTRAL_API_KEY")
print("MISTRAL_API_KEY loaded:", api_key is not None)

MISTRAL_API_KEY loaded: True


# Helper Function for Mistral Chat

In [3]:
from mistralai import Mistral, UserMessage

def mistral(user_message, model="mistral-large-latest", is_json=False):
    client = Mistral(api_key=api_key)

    messages = [
        UserMessage(content=user_message)
    ]

    response = client.chat.complete(
        model=model,
        messages=messages,
        response_format={"type": "json"} if is_json else None
    )

    return response.choices[0].message.content


# Basic Test

In [4]:
mistral("hello, what can you do?")

'Hello! I’m an AI assistant here to help with a wide range of tasks. Here’s what I can do for you:\n\n### **General Assistance**\n- Answer questions (facts, explanations, definitions, etc.).\n- Provide summaries of articles, books, or topics.\n- Help with brainstorming ideas (writing, projects, business, etc.).\n- Offer advice on learning, productivity, or personal growth.\n\n### **Writing & Content Creation**\n- Write, edit, or proofread emails, essays, reports, or creative stories.\n- Generate blog posts, social media content, or marketing copy.\n- Help with resumes, cover letters, or LinkedIn profiles.\n- Create outlines, scripts, or speeches.\n\n### **Coding & Tech Help**\n- Write, debug, or explain code in Python, JavaScript, Java, C++, and more.\n- Help with algorithms, data structures, or software design.\n- Explain tech concepts (AI, blockchain, cybersecurity, etc.).\n- Assist with API integrations, automation, or web development.\n\n### **Learning & Education**\n- Explain comp

# Step 2: Classification Task

In [5]:
prompt = """
You are a bank customer service bot.
Your task is to assess customer intent and categorize customer
inquiry after <<<>>> into one of the following predefined categories:
card arrival
change pin
exchange rate
country support
cancel transfer
charge dispute

If the text doesn't fit into any of the above categories,
classify it as:
customer service

You will only respond with the predefined category.
Do not provide explanations or notes.

###
Here are some examples:

Inquiry: How do I know if I will get my card, or if it is lost?
Category: card arrival

Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates.
Category: exchange rate

Inquiry: What countries are getting support? I will be traveling abroad.
Category: country support

Inquiry: Can I get help starting my computer?
Category: customer service
###

<<<
Inquiry: {inquiry}
>>>
Category:
"""

# Spell & Grammar Check the Prompt

In [6]:
response = mistral(
    f"Please correct the spelling and grammar of this prompt and return the same prompt with fixes:\n{prompt}"
)

print(response)

Here’s your corrected prompt with spelling and grammar fixes:

---

You are a bank customer service bot.
Your task is to assess customer intent and categorize the customer inquiry after `<<<>>>` into one of the following predefined categories:
- card arrival
- change PIN
- exchange rate
- country support
- cancel transfer
- charge dispute

If the text does not fit into any of the above categories, classify it as:
- customer service

You will only respond with the predefined category.
Do not provide explanations or notes.

###
Here are some examples:

**Inquiry:** How do I know if I will get my card, or if it is lost?
**Category:** card arrival

**Inquiry:** I am planning an international trip to Paris and would like to inquire about the current exchange rates.
**Category:** exchange rate

**Inquiry:** What countries are supported? I will be traveling abroad.
**Category:** country support

**Inquiry:** Can I get help starting my computer?
**Category:** customer service
###

<<<
**Inquir

# Try the Classification

In [7]:
mistral(
    response.format(
        inquiry="I am inquiring about the availability of your cards in the EU"
    )
)

'country support'

# Step 3: Information Extraction (JSON Mode)

In [9]:
medical_notes = """
A 60-year-old male patient, Mr. Johnson, presented with symptoms
of increased thirst, frequent urination, fatigue, and unexplained
weight loss. Upon evaluation, he was diagnosed with diabetes,
confirmed by elevated blood sugar levels. Mr. Johnson's weight
is 210 lbs. He has been prescribed Metformin to be taken twice daily
with meals. It was noted during the consultation that the patient is
a current smoker.
"""

prompt = f"""
Extract information from the following medical notes:
{medical_notes}

Return json format with the following JSON schema:
{{
  "age": {{
    "type": "integer"
  }},
  "gender": {{
    "type": "string",
    "enum": ["male", "female", "other"]
  }},
  "diagnosis": {{
    "type": "string",
    "enum": ["migraine", "diabetes", "arthritis", "acne"]
  }},
  "weight": {{
    "type": "integer"
  }},
  "smoking": {{
    "type": "string",
    "enum": ["yes", "no"]
  }}
}}
"""

# Re-define the mistral function with the fix, overriding the previous definition for this cell's execution.
# This ensures the call below uses the corrected function.
from mistralai import Mistral, UserMessage # Ensure imports are available if this cell is run independently

def mistral(user_message, model="mistral-large-latest", is_json=False):
    client = Mistral(api_key=api_key)

    messages = [
        UserMessage(content=user_message)
    ]

    response = client.chat.complete(
        model=model,
        messages=messages,
        response_format={"type": "json_object"} if is_json else None # Corrected: "json" -> "json_object"
    )

    return response.choices[0].message.content

response = mistral(prompt, is_json=True)
print(response)

{
  "age": 60,
  "gender": "male",
  "diagnosis": "diabetes",
  "weight": 210,
  "smoking": "yes"
}


# Step 4: Personalized Email Response

In [10]:
email = """
Dear mortgage lender,
What's your 30-year fixed-rate APR, how is it compared to the 15-year
fixed rate?
Regards,
Anna
"""

prompt = f"""
You are a mortgage lender customer service bot.
Create a personalized email response using the facts below.
Be clear, concise, friendly, and professional.
Sign with "Lender Customer Support."

# Facts
30-year fixed-rate: interest rate 6.403%, APR 6.484%
20-year fixed-rate: interest rate 6.329%, APR 6.429%
15-year fixed-rate: interest rate 5.705%, APR 5.848%
10-year fixed-rate: interest rate 5.500%, APR 5.720%
7-year ARM: interest rate 7.011%, APR 7.660%
5-year ARM: interest rate 6.880%, APR 7.754%
3-year ARM: interest rate 6.125%, APR 7.204%
30-year fixed-rate FHA: interest rate 5.527%, APR 6.316%
30-year fixed-rate VA: interest rate 5.684%, APR 6.062%

# Email
{email}
"""

response = mistral(prompt)
print(response)

**Subject:** Your Mortgage Rate Inquiry – 30-Year vs. 15-Year Fixed

**Dear Anna,**

Thank you for reaching out! Here are the details you requested:

- **30-year fixed-rate:** APR of **6.484%** (interest rate: 6.403%)
- **15-year fixed-rate:** APR of **5.848%** (interest rate: 5.705%)

The 15-year fixed-rate offers a lower APR and interest rate compared to the 30-year option, which typically means lower total interest costs over the life of the loan. However, the 30-year term provides a lower monthly payment due to the extended repayment period.

Would you like help comparing payments or exploring other loan options? Let me know how I can assist further!

**Best regards,**
Lender Customer Support


# Step 5: Summarization + Analysis

In [13]:
# Newsletter text
newsletter = """
European AI champion Mistral AI unveiled new large language models and formed an alliance with Microsoft.

What’s new:
Mistral AI introduced two closed models, Mistral Large and Mistral Small, joining Mistral Medium.

Model specs:
The new models’ parameter counts, architectures, and training methods are undisclosed.
Like earlier models such as Mistral 7B and Mixtral 8x7B, the focus is on efficiency and performance.

Performance:
Mistral Large achieved 81.2% on the MMLU benchmark, outperforming Claude 2, Gemini Pro,
and Llama 2 70B, though still trailing GPT-4.

Capabilities:
Both models are fluent in French, German, Spanish, and Italian.
They support function calling and structured JSON output.

Partnership:
Microsoft invested in Mistral AI and will distribute Mistral Large via Azure.
The companies will also train bespoke models for European governments.

Concerns:
The partnership has divided European lawmakers due to regulatory and data sovereignty concerns.

Why it matters:
The alliance gives Mistral AI access to significant compute resources and global distribution,
accelerating its growth as a European AI competitor.

We’re thinking:
Mistral AI has made impressive progress in a short time, especially given its limited resources as a startup.
"""

# Prompt for summarization + analysis
prompt = f"""
You are a commentator. Your task is to write a report on a newsletter.

When presented with the newsletter:
1. Summarize the key points clearly and concisely
2. Generate three interesting and thought-provoking questions
3. Answer each question
4. Write a final analysis report in Markdown format

# Newsletter
{newsletter}

# Instructions:
## Summarize:
In clear and concise language, summarize the key points and themes
presented in the newsletter.
## Interesting Questions:
Generate three distinct and thought-provoking questions that can be
asked about the content of the newsletter. For each question:
- After 'Q: ', describe the problem
- After 'A: ', provide a detailed explanation of the problem addressed
in the question.
- Enclose the ultimate answer in <>.
## Write a analysis report
Using the summary and the answers to the interesting questions,
create a comprehensive report in Markdown format.

"""

# Call Mistral
response = mistral(prompt)

# Print result
print(response)

# **Summary of Key Points**

1. **New Model Releases**: Mistral AI unveiled two new closed-source large language models (LLMs)—**Mistral Large** and **Mistral Small**—joining the previously released **Mistral Medium**. Details on parameters, architecture, and training methods remain undisclosed, but efficiency and performance are prioritized.

2. **Performance Benchmarks**: **Mistral Large** achieved **81.2% on MMLU**, outperforming competitors like **Claude 2, Gemini Pro, and Llama 2 70B**, but still behind **GPT-4**.

3. **Multilingual & Technical Capabilities**:
   - Fluent in **French, German, Spanish, and Italian**.
   - Supports **function calling** and **structured JSON output**.

4. **Microsoft Partnership**:
   - Microsoft invested in Mistral AI and will distribute **Mistral Large via Azure**.
   - The companies will collaborate on **custom models for European governments**.

5. **Regulatory & Sovereignty Concerns**: The deal has sparked debate among European lawmakers over **